In [4]:
library(tidyverse)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [5]:
bike <- read.csv('https://raw.githubusercontent.com/IAA-Faculty/statistical_foundations/master/bike.csv')

1. Run the following code to get the training and test split:

In [6]:
set.seed(123) 
bike <- bike %>% mutate(id = row_number()) 
train <- bike %>% sample_frac(0.7)
test <- anti_join(bike, train, by = "id")

2. There are abnormal times where the number of casual users is greater than or equal to the number of registered users. You can use the following code to create a variable casual_high that captures this

In [7]:
train$casual_high <- train$casual >= train$registered

In [11]:
train  %>% str()

'data.frame':	12165 obs. of  18 variables:
 $ dteday     : int  15103 15055 15119 15465 15177 15260 15653 15093 15506 15363 ...
 $ season     : int  2 2 2 2 3 4 4 2 2 1 ...
 $ yr         : int  0 0 0 1 0 0 1 0 1 1 ...
 $ mnth       : int  5 3 5 5 7 10 11 4 6 1 ...
 $ hr         : int  7 11 8 2 6 15 17 18 16 14 ...
 $ holiday    : int  0 0 0 0 0 0 0 0 0 0 ...
 $ weekday    : int  1 2 3 6 5 4 5 5 5 2 ...
 $ workingday : int  1 1 1 0 1 1 1 1 1 1 ...
 $ weathersit : int  1 1 1 1 1 1 1 1 1 1 ...
 $ temp       : num  0.48 0.44 0.64 0.58 0.76 0.66 0.42 0.52 0.72 0.36 ...
 $ atemp      : num  0.47 0.439 0.606 0.545 0.742 ...
 $ hum        : num  0.63 0.51 0.69 0.88 0.75 0.78 0.44 0.48 0.37 0.81 ...
 $ windspeed  : num  0.2537 0.3284 0.2239 0 0.0896 ...
 $ casual     : int  16 22 36 24 13 27 61 55 132 35 ...
 $ registered : int  221 64 495 51 83 128 593 384 426 138 ...
 $ cnt        : int  237 86 531 75 96 155 654 439 558 173 ...
 $ id         : int  2986 1842 3371 11638 4761 6746 16128 2757 12

In [12]:
train <- train  %>% mutate(across(c(season, yr, mnth, hr, holiday, weekday, workingday, weathersit), as.factor))

ERROR: Error in eval(expr, envir, enclos): object '.' not found


In [17]:
bike_logit <- glm(casual_high ~ . -season - cnt - casual - registered - id - dteday, data = train, family = binomial(link = "logit"))


In [16]:
bike_logit  %>% summary()


Call:
glm(formula = casual_high ~ . - cnt - casual - registered - id - 
    dteday, family = binomial(link = "logit"), data = train)

Coefficients: (1 not defined because of singularities)
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -4.1726     0.6174  -6.758 1.40e-11 ***
season2       -0.3381     0.3869  -0.874 0.382210    
season3       -1.1607     0.6317  -1.837 0.066159 .  
season4       -2.0484     0.5698  -3.595 0.000324 ***
yr1           -0.9707     0.1369  -7.088 1.36e-12 ***
mnth2         -0.4161     0.4501  -0.924 0.355325    
mnth3          0.9741     0.3754   2.595 0.009462 ** 
mnth4          1.1471     0.5478   2.094 0.036278 *  
mnth5          1.0465     0.5846   1.790 0.073418 .  
mnth6         -0.2127     0.6467  -0.329 0.742247    
mnth7          1.2257     0.8058   1.521 0.128257    
mnth8          0.3263     0.8261   0.395 0.692875    
mnth9          1.4781     0.7487   1.974 0.048353 *  
mnth10         1.7537     0.7081   2.477 0.013267 *  


In [20]:
full.model <- glm(casual_high ~ . - cnt - casual - registered - id - dteday, data = train, family = binomial(link = "logit"))
empty.model <- glm(casual_high ~ 1, data = train, family = binomial(link = "logit"))

back.model <- step(full.model, 
                   scope = list(lower = formula(empty.model), 
                                upper = formula(full.model)), 
                                direction = "backward", 
                                k = qchisq(.001, 1, lower.tail = FALSE))

Start:  AIC=2672.65
casual_high ~ (dteday + season + yr + mnth + hr + holiday + weekday + 
    workingday + weathersit + temp + atemp + hum + windspeed + 
    casual + registered + cnt + id) - cnt - casual - registered - 
    id - dteday


Step:  AIC=2672.65
casual_high ~ season + yr + mnth + hr + holiday + weekday + weathersit + 
    temp + atemp + hum + windspeed

             Df Deviance    AIC
- mnth       11   2156.1 2610.9
- weathersit  3   2105.3 2646.6
- season      3   2115.3 2656.7
- windspeed   1   2099.3 2662.3
- temp        1   2099.4 2662.5
- hum         1   2100.3 2663.3
- atemp       1   2101.2 2664.2
<none>            2098.8 2672.7
- holiday     1   2117.0 2680.0
- weekday     6   2171.4 2680.3
- yr          1   2153.1 2716.1
- hr         23   2452.4 2777.2

Step:  AIC=2610.89
casual_high ~ season + yr + hr + holiday + weekday + weathersit + 
    temp + atemp + hum + windspeed

             Df Deviance    AIC
- weathersit  3   2162.3 2584.6
- windspeed   1   2156.1 260